In [1]:
import os
import io
import base64
import urllib.parse
from rdkit import Chem
from rdkit.Chem import AllChem
from PIL import Image
import py3Dmol
from IPython.display import display, HTML



In [2]:

def encode(smile):
    return urllib.parse.quote(smile, safe="")

def decode(filename):
    return urllib.parse.unquote(filename)

def show_image(filepath):
    if not os.path.exists(filepath):
        print(f"File không tồn tại: {filepath}")
        return
    
    try:
        # Đọc ảnh
        img = mpimg.imread(filepath)
        
        # Hiển thị ảnh
        plt.imshow(img)
        plt.axis('off')  # tắt trục
        plt.show()
        
    except Exception as e:
        print(f"Lỗi khi hiển thị ảnh: {e}")


In [3]:
def save_smiles_3d(smiles, save_dir="outputs/molecules/3d",
                              filename=None,
                              width=600,
                              height=500,
                              style="stick",   # "stick", "ball", "ball+stick", "line", "sphere"
                              background="#ffffff",
                              save_png=True,
                              fallback_to_rdkit=True,
                              return_view=False):
    """
    Tạo interactive 3D viewer bằng py3Dmol từ SMILES. 
    Nếu save_png=True, cố gắng lưu ảnh PNG (dùng view.png()).
    Trả về đường dẫn file PNG (nếu lưu thành công) hoặc viewer object nếu return_view=True.
    """
    os.makedirs(save_dir, exist_ok=True)
    if filename is None:
        filename = encode(smiles)
    png_path = os.path.join(save_dir, f"{filename}.png")

    # Nếu file đã tồn tại -> hiển thị và trả về
    if os.path.exists(png_path):
        print(f"Đã có file: {png_path}")
        show_image(png_path)
        if return_view:
            # vẫn trả viewer sau
            pass
        return png_path

    # --- 1) SMILES -> Mol, tạo 3D bằng ETKDG + optimize ---
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        raise ValueError(f"SMILES không hợp lệ: {smiles}")
    mol_h = Chem.AddHs(mol)
    params = AllChem.ETKDGv3()
    params.randomSeed = 42
    emb_ok = AllChem.EmbedMolecule(mol_h, params)
    if emb_ok != 0:
        # Try again if embedding fails
        AllChem.EmbedMolecule(mol_h, params)
    try:
        AllChem.UFFOptimizeMolecule(mol_h)
    except Exception:
        try:
            AllChem.MMFFOptimizeMolecule(mol_h)
        except Exception:
            pass

    # Convert to PDB block (string) for py3Dmol
    pdb_block = Chem.MolToPDBBlock(mol_h)

    # --- 2) Tạo viewer py3Dmol ---
    view = py3Dmol.view(width=width, height=height)
    view.addModel(pdb_block, "pdb")

    # Style parsing
    # support "ball", "stick", "ball+stick", "line", "sphere"
    style_obj = {}
    if style == "stick":
        style_obj = {"stick": {}}
    elif style == "ball":
        style_obj = {"sphere": {"radius": 0.4}}
    elif style == "ball+stick" or style == "ball-stick":
        style_obj = {"stick": {}, "sphere": {"scale": 0.3}}
    elif style == "line":
        style_obj = {"line": {}}
    elif style == "sphere":
        style_obj = {"sphere": {}}
    else:
        style_obj = {"stick": {}}

    view.setStyle({}, style_obj)
    view.setBackgroundColor(background)
    view.zoomTo()
    # Hiển thị interactive trong notebook
    try:
        display(view)
    except Exception:
        # fallback: inject HTML (py3Dmol usually displays fine)
        display(HTML(view._make_html()))

    # --- 3) Nếu save_png True, cố gắng lấy PNG từ viewer ---
    if save_png:
        png_saved = False
        try:
            # view.png() trả về bytes hoặc data url -> thử xử lý
            png_data = view.png()  # many envs return a base64 string "data:image/png;base64,...."
            if isinstance(png_data, bytes):
                # trực tiếp bytes
                with open(png_path, "wb") as f:
                    f.write(png_data)
                png_saved = True
            elif isinstance(png_data, str):
                # có thể là "data:image/png;base64,...."
                if png_data.startswith("data:image"):
                    header, b64 = png_data.split(",", 1)
                    img_bytes = base64.b64decode(b64)
                    with open(png_path, "wb") as f:
                        f.write(img_bytes)
                    png_saved = True
                else:
                    # một số triển khai trả về base64 không có header
                    try:
                        img_bytes = base64.b64decode(png_data)
                        with open(png_path, "wb") as f:
                            f.write(img_bytes)
                        png_saved = True
                    except Exception:
                        png_saved = False
        except Exception as e:
            # view.png() thất bại trong một số môi trường
            # print("view.png() thất bại:", e)
            png_saved = False

        # --- Fallback: nếu không lấy được từ py3Dmol, dùng RDKit 2D/3D render ---
        if not png_saved and fallback_to_rdkit:
            try:
                # Render 3D snapshot bằng rdkit's MolDraw2DCairo (dùng phối cảnh 3D)
                from rdkit.Chem.Draw import rdMolDraw2D
                drawer = rdMolDraw2D.MolDraw2DCairo(width, height)
                opts = drawer.drawOptions()
                opts.is3D = True
                rdMolDraw2D.PrepareAndDrawMolecule(drawer, mol_h)
                drawer.FinishDrawing()
                img = Image.open(io.BytesIO(drawer.GetDrawingText()))
                img.save(png_path)
                png_saved = True
            except Exception as e2:
                print("Fallback RDKit render failed:", e2)
                png_saved = False

        if png_saved:
            print("Đã lưu PNG:", png_path)
            show_image(png_path)
            if return_view:
                return view
            return png_path
        else:
            print("Không lưu được PNG từ py3Dmol; viewer đã hiển thị interactive (nếu notebook).")
            if return_view:
                return view
            return None
    else:
        # không lưu, chỉ show
        if return_view:
            return view
        return None

In [5]:
smiles_test = "CCOc1ccc2nc(S(N)(=O)=O)sc2c1"
save_smiles_3d(smiles_test)

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

Fallback RDKit render failed: Cannot set unknown attribute 'is3D'
Không lưu được PNG từ py3Dmol; viewer đã hiển thị interactive (nếu notebook).
